# 利用CNN实现cifar10 的图像识别任务

### 1. 下载 cifar10 数据


In [3]:
import cifar10, cifar10_input
cifar10.maybe_download_and_extract()

>> Downloading cifar-10-binary.tar.gz 100.0%
Successfully downloaded cifar-10-binary.tar.gz 170052171 bytes.


### 2. 数据预处理

In [6]:
import tensorflow as tf
import numpy as np
import time

In [9]:
max_steps = 3000
batch_size=128
data_dir = '/tmp/cifar10_data/cifar-10-batches-bin' # cifar10默认被下载到这个位置

生成训练数据和测试数据，其中训练数据使用了数据增强技术

In [11]:
images_train, labels_train = cifar10_input.distorted_inputs(data_dir=data_dir, batch_size=batch_size)
images_test, labels_test = cifar10_input.inputs(eval_data=True, data_dir=data_dir, batch_size=batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


### 3. 构建CNN

In [12]:
def variable_with_weight_loss(shape, stddev, wl):
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if wl is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var),  wl, name="weight_loss")
        tf.add_to_collection('losses', weight_loss)
    return var

In [13]:
image_holder = tf.placeholder(tf.float32, [batch_size, 24,24, 3])
label_holder = tf.placeholder(tf.int32, [batch_size])

构建层结构

In [ ]:
# 第一层 卷积层
weight1 = variable_with_weight_loss(shape=[5,5,3,64], stddev=5e-2, wl=0.0)
bias1 = tf.Variable(tf.constant(0.0, shape=[64]))
conv1 = tf.nn.conv2d(image_holder, weight1, strides=[1,1,1,1], padding='SAME')
conv1 = tf.nn.relu(tf.nn.bias_add(conv1, bias1))
pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')
norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)

# 第二层 卷积层
weight2 = variable_with_weight_loss(shape=[5,5,64,64], stddev=5e-2, wl=0.0)
bias2 =  tf.Variable(tf.constant(0.1, shape=[64]))
conv2 = tf.nn.conv2d(norm1, weight2, [1,1,1,1], padding='SAME')
conv2 = tf.nn.relu(tf.nn.bias_add(conv2, bias2))
norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)
pool2 = tf.nn.max_pool(norm2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

# 第三层 全连接层
reshape = tf.reshape(pool2, [batch_size,-1])
dim = reshape.get_shape()[1].value
weight3 =variable_with_weight_loss(shape=[dim, 384], stddev=0.04, wl=0.004)
bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)

# 第四层  全连接层
weight4 = variable_with_weight_loss(shape=[384, 192], stddev=0.04, wl=0.004)
bias4 = tf.Variable(tf.constant(0.1, shape=[192]))
local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)

# 第五层  输出层
weight5 = variable_with_weight_loss(shape=[192,10], stddev=1/192.0, wl=0.0)
bias5 = tf.Variable(tf.constant(0.0, shape=[10]))
logits = tf.add(tf.matmul(local4, weight5), bias5)

计算loss

In [18]:
def loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels,name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy,name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)
    
    return tf.add_n(tf.get_collection('losses'), name='total_loss')


In [20]:
loss = loss(logits, label_holder)
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [21]:
top_k_op = tf.nn.in_top_k(logits, label_holder, 1)

训练

In [22]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [23]:
tf.train.start_queue_runners()

[<Thread(Thread-4, started daemon 140313924466432)>,
 <Thread(Thread-5, started daemon 140313916073728)>,
 <Thread(Thread-6, started daemon 140313762985728)>,
 <Thread(Thread-7, started daemon 140313754593024)>,
 <Thread(Thread-8, started daemon 140313746200320)>,
 <Thread(Thread-9, started daemon 140313737807616)>,
 <Thread(Thread-10, started daemon 140313729414912)>,
 <Thread(Thread-11, started daemon 140313721022208)>,
 <Thread(Thread-12, started daemon 140313712629504)>,
 <Thread(Thread-13, started daemon 140313427441408)>,
 <Thread(Thread-14, started daemon 140313419048704)>,
 <Thread(Thread-15, started daemon 140313410656000)>,
 <Thread(Thread-16, started daemon 140313402263296)>,
 <Thread(Thread-17, started daemon 140313393870592)>,
 <Thread(Thread-18, started daemon 140313385477888)>,
 <Thread(Thread-19, started daemon 140313377085184)>,
 <Thread(Thread-20, started daemon 140313368692480)>,
 <Thread(Thread-21, started daemon 140313360299776)>,
 <Thread(Thread-22, started daemon

In [25]:
for step in range(max_steps):
    start_time = time.time()
    image_batch,label_batch =  sess.run([images_train, labels_train])
    _, loss_val = sess.run([train_op, loss], feed_dict={image_holder:image_batch, label_holder:label_batch})
    duration = time.time() - start_time
    
    if step%10 == 0:
        examples_per_sec = batch_size/duration
        sec_per_batch = float(duration)
        format_str = ('step %d, loss = %.2f (%.1f examples/sec; %.3f sec/batch)')
        print(format_str%(step, loss_val, examples_per_sec, sec_per_batch))

step 0, loss = 7.05 (52.0 examples/sec; 2.462 sec/batch)
step 10, loss = 5.14 (71.1 examples/sec; 1.800 sec/batch)
step 20, loss = 3.91 (74.5 examples/sec; 1.718 sec/batch)
step 30, loss = 3.30 (70.9 examples/sec; 1.805 sec/batch)
step 40, loss = 2.74 (71.4 examples/sec; 1.792 sec/batch)
step 50, loss = 2.33 (64.0 examples/sec; 1.999 sec/batch)
step 60, loss = 2.20 (45.7 examples/sec; 2.801 sec/batch)
step 70, loss = 2.13 (67.4 examples/sec; 1.899 sec/batch)
step 80, loss = 2.17 (66.8 examples/sec; 1.915 sec/batch)
step 90, loss = 1.80 (42.1 examples/sec; 3.037 sec/batch)
step 100, loss = 1.95 (77.2 examples/sec; 1.658 sec/batch)
step 110, loss = 1.97 (56.0 examples/sec; 2.287 sec/batch)
step 120, loss = 1.78 (41.6 examples/sec; 3.081 sec/batch)
step 130, loss = 1.91 (58.1 examples/sec; 2.204 sec/batch)
step 140, loss = 1.94 (53.6 examples/sec; 2.388 sec/batch)
step 150, loss = 1.82 (53.0 examples/sec; 2.413 sec/batch)
step 160, loss = 1.88 (64.9 examples/sec; 1.974 sec/batch)
step 170

test

In [26]:
num_examples = 10000
import math
num_iter = int(math.ceil(num_examples/batch_size))
true_count = 0
total_sample_count = num_iter*batch_size
step = 0
while step <num_iter:
    image_batch, label_batch = sess.run([images_testst, labels_test])
    predictions =sess.run([top_k_op], feed_dictimage_holderolder:image_balabel_holderl_holder:label_batch})
    true_count +=%precision.sum(predictions)
    step += 10000

In [27]:
precision = true_count/total_sample_count
print('precision @1=%.3f'%precision)

precision @1=0.711
